# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
%matplotlib inline

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_to_load = ("city_data.csv")

In [3]:
city_data = pd.read_csv(file_to_load)

In [4]:
city_data.head()

,Unnamed: 0,City,Country,Date,Humidity,Cloudiness,Lat,Long,Temp,WindSpeed
0,0,El Puerto de Santa María,ES,1624782280,77,75,36.5939,-6.2330,73.51,8.05
1,1,Pavlivka,UA,1624782197,80,67,49.4414,28.4596,68.41,13.29
2,2,Avera,US,1624782470,97,96,33.1940,-82.5271,75.07,4.92
3,3,Rikitea,PF,1624782207,78,7,-23.1203,-134.9692,74.25,11.86
4,4,Ushuaia,AR,1624782470,81,90,-54.8000,-68.3000,38.86,10.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

In [6]:
locations = city_data[["Lat", "Long"]]
weights = city_data["Humidity"]

In [7]:
 #Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
heat_layer.max_intensity = 100
heat_layer.point_radius = 5

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# A max temperature lower than 80 degrees but higher than 70.
temp_city_data = city_data.loc[(city_data["Temp"] > 70) & (city_data["Temp"] < 80)]
temp_city_data.head()

,Unnamed: 0,City,Country,Date,Humidity,Cloudiness,Lat,Long,Temp,WindSpeed
0,0,El Puerto de Santa María,ES,1624782280,77,75,36.5939,-6.2330,73.51,8.05
2,2,Avera,US,1624782470,97,96,33.1940,-82.5271,75.07,4.92
3,3,Rikitea,PF,1624782207,78,7,-23.1203,-134.9692,74.25,11.86
5,5,Albany,US,1624782471,90,88,42.6001,-73.9662,75.81,5.66
6,6,Hilo,US,1624782471,88,1,19.7297,-155.0900,73.80,3.44


In [10]:
# Wind speed less than 10 mph.
wind_city_data = city_data.loc[city_data["WindSpeed"] < 10]
wind_city_data.head()

,Unnamed: 0,City,Country,Date,Humidity,Cloudiness,Lat,Long,Temp,WindSpeed
0,0,El Puerto de Santa María,ES,1624782280,77,75,36.5939,-6.2330,73.51,8.05
2,2,Avera,US,1624782470,97,96,33.1940,-82.5271,75.07,4.92
5,5,Albany,US,1624782471,90,88,42.6001,-73.9662,75.81,5.66
6,6,Hilo,US,1624782471,88,1,19.7297,-155.0900,73.80,3.44
7,7,Port,CD,1624782395,42,2,-7.0349,29.7638,78.84,5.50


In [11]:
# Zero cloudiness.
cloudiness_city_data = city_data.loc[city_data["Cloudiness"] == 0]
cloudiness_city_data.head()

,Unnamed: 0,City,Country,Date,Humidity,Cloudiness,Lat,Long,Temp,WindSpeed
12,12,Semnan,IR,1624782473,11,0,35.5729,53.3971,93.27,7.83
18,18,Susanville,US,1624782475,40,0,40.4163,-120.6530,71.10,1.01
44,44,Kifrī,IQ,1624782486,6,0,34.6959,44.9661,111.04,5.79
57,57,Paralímni,CY,1624782490,88,0,35.0375,33.9833,95.49,15.01
68,68,Bilma,NE,1624782494,9,0,18.6853,12.9164,97.95,12.48


In [12]:
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
niceweather_city_data = city_data.loc[(city_data["Temp"] > 70) & (city_data["Temp"] < 80) & (city_data["WindSpeed"] < 10) & (city_data["Cloudiness"] == 0)]
niceweather_city_data

,Unnamed: 0,City,Country,Date,Humidity,Cloudiness,Lat,Long,Temp,WindSpeed
18,18,Susanville,US,1624782475,40,0,40.4163,-120.6530,71.10,1.01
96,96,Tautira,PF,1624782342,79,0,-17.7333,-149.1500,77.02,7.02
104,104,Chiredzi,ZW,1624782508,22,0,-21.0500,31.6667,75.90,3.76
115,115,Beloha,MG,1624782367,41,0,-25.1667,45.0500,75.99,5.35
168,168,Mossamedes,AO,1624782324,41,0,-15.1961,12.1522,77.47,7.49
264,264,Coos,US,1624782566,96,0,43.1665,-124.0015,72.64,1.48
270,270,Vangaindrano,MG,1624782568,66,0,-23.3500,47.6000,73.63,6.44
286,286,Grootfontein,NaN,1624782575,26,0,-19.5667,18.1167,71.17,6.91
508,508,Kuopio,FI,1624782638,68,0,62.8924,27.6770,73.35,5.75
537,537,Hervey,US,1624782672,71,0,33.4982,-93.7771,76.62,7.72


In [13]:
city_data.count()

Unnamed: 0    568
City          568
Country       565
Date          568
Humidity      568
Cloudiness    568
Lat           568
Long          568
Temp          568
WindSpeed     568
dtype: int64

In [14]:
# Drop any rows will null values.
no_null_city_data = city_data.dropna(how='any')
no_null_city_data.count()

Unnamed: 0    565
City          565
Country       565
Date          565
Humidity      565
Cloudiness    565
Lat           565
Long          565
Temp          565
WindSpeed     565
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = niceweather_city_data.loc[:,["City","Country","Lat","Long"]]
hotel_df

,City,Country,Lat,Long
18,Susanville,US,40.4163,-120.6530
96,Tautira,PF,-17.7333,-149.1500
104,Chiredzi,ZW,-21.0500,31.6667
115,Beloha,MG,-25.1667,45.0500
168,Mossamedes,AO,-15.1961,12.1522
264,Coos,US,43.1665,-124.0015
270,Vangaindrano,MG,-23.3500,47.6000
286,Grootfontein,NaN,-19.5667,18.1167
508,Kuopio,FI,62.8924,27.6770
537,Hervey,US,33.4982,-93.7771


In [16]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Long,Hotel Name
18,Susanville,US,40.4163,-120.6530,
96,Tautira,PF,-17.7333,-149.1500,
104,Chiredzi,ZW,-21.0500,31.6667,
115,Beloha,MG,-25.1667,45.0500,
168,Mossamedes,AO,-15.1961,12.1522,
264,Coos,US,43.1665,-124.0015,
270,Vangaindrano,MG,-23.3500,47.6000,
286,Grootfontein,NaN,-19.5667,18.1167,
508,Kuopio,FI,62.8924,27.6770,
537,Hervey,US,33.4982,-93.7771,


In [20]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

In [24]:
# sample url from google:
# https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY
# query_url = base_url + "location=" + lat + "," + long + "&radius=" + 5000 + "&type=" + type

# Set parameters to search for hotels with 5000 meters.
locate = hotel_df[["Lat", "Long"]]

parameters = {
    "key" : g_key,
    "keyword" : "hotel",
    "radius" : 5000,
    "type" : "hotel"
}

hotel_name = []

for index, row in hotel_df.iterrows()
    response = requests.get(query_url + city).json()
    lat.append(response['coord']['lat'])
    temp.append(response['main']['temp'])
    
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.

SyntaxError: invalid syntax (<ipython-input-24-242adb129478>, line 6)

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [19]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))